In [1]:
conda install -c conda-forge notebook pandas sqlalchemy snowflake-sqlalchemy


Retrieving notices: ...working... done
Channels:
 - conda-forge
 - defaults
Platform: osx-64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - notebook
    - pandas
    - snowflake-sqlalchemy
    - sqlalchemy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    asn1crypto-1.5.1           |     pyhd8ed1ab_1          84 KB  conda-forge
    ca-certificates-2024.12.14 |       h8857fd0_0         153 KB  conda-forge
    certifi-2024.12.14         |     pyhd8ed1ab_0         158 KB  conda-forge
    conda-24.11.3              |  py312hb401068_0         1.1 MB  conda-forge
    importlib_resources-6.5.2  |     pyhd8ed1ab_0          33 KB  conda-forge
    jupyterlab-4.3.4           |     pyhd8ed1ab_0         6.9 MB  conda-forge
    jupyterlab_server-2.27.3   |     pyhd8ed1ab_1          48 KB  conda-forge
    libcxx-19.1.7              

In [1]:
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import pandas as pd
import uuid

import os

user = os.environ.get('SNOWFLAKE_USER')
password = os.environ.get('SNOWFLAKE_PASS')
account = os.environ.get('SNOWFLAKE_ACCOUNT')

In [5]:
if not all([user, password, account]):
    print("Missing environment variables. Please check Snowflake Connection setup.")
else:
    from sqlalchemy import create_engine
    from snowflake.sqlalchemy import URL

    engine = create_engine(URL(
        account = account,
        user = user,
        password = password,
        database = 'SF_LEADMANAGEMENT',
        schema = 'PUBLIC',
        warehouse = 'COMPUTE_WH',
        role='ACCOUNTADMIN'
    ))

    connection = engine.connect()

In [7]:
# Read data from CompanyLeads
query = "SELECT * FROM CompanyLeads;"
df_leads = pd.read_sql(query, connection)

print(df_leads.head())

                                     id  state      createddateutc  \
0  d73c7d93-bc0b-c284-8786-08db8cec307b      3 2023-07-25 08:50:26   
1  f4a6f637-b7e9-c4de-8339-08db8cef5067      2 2023-07-25 09:12:48   
2  4633aa15-3c41-cc94-9176-08db8d0f3c4b      1 2023-07-25 13:01:18   
3  57e80bf5-492d-cf3e-a516-08db8d179419      2 2023-07-25 14:01:01   
4  1a527929-d64c-c031-104d-08db8d27c11a      2 2023-07-25 15:56:49   

  cancellationrequestdateutc cancellationdateutc cancellationrejectiondateutc  \
0        2023-07-31 12:21:42                 NaT          2023-08-01 10:25:30   
1        2023-07-31 13:56:10 2023-08-01 10:24:21                          NaT   
2        2023-08-01 10:19:09                 NaT                          NaT   
3        2023-08-01 06:51:10 2023-08-01 10:23:46                          NaT   
4        2023-08-01 07:11:25 2023-08-01 10:24:38                          NaT   

         soldemployee cancelledemployee      updateddateutc  
0  xyz@pflegehilfe.de         

In [76]:
def transform_data(df):
    events = []
    for index, row in df.iterrows():
        # Event when lead is sold, State = 0
        if pd.notnull(row['createddateutc']):
            events.append({
                'id': str(uuid.uuid4()),
                'eventtype': 'LeadSold',
                'eventemployee': row['soldemployee'],
                'eventdate': row['createddateutc'],
                'leadid': row['id'],
                'updateddateutc': row['updateddateutc']
            })

        # Event when cancellation is requested, State = 1
        if pd.notnull(row['cancellationrequestdateutc']):
            events.append({
                'id': str(uuid.uuid4()),
                'eventtype': 'LeadRequestedCancellation',
                'eventemployee': 'Unknown',  # As specified in pdf use "Unknown" for these events
                'eventdate': row['cancellationrequestdateutc'],
                'leadid': row['id'],
                'updateddateutc': row['updateddateutc']
            })

        # Event when lead is cancelled, State = 2
        if pd.notnull(row['cancellationdateutc']):
            events.append({
                'id': str(uuid.uuid4()),
                'eventtype': 'LeadCancelled',
                'eventemployee': row['cancelledemployee'],
                'eventdate': row['cancellationdateutc'],
                'leadid': row['id'],
                'updateddateutc': row['updateddateutc']
            })

        # Event when cancellation is rejected, State = 3
        if pd.notnull(row['cancellationrejectiondateutc']):
            events.append({
                'id': str(uuid.uuid4()),
                'eventtype': 'LeadCancellationRejected',
                'eventemployee': 'Unknown',  # As specified, use "Unknown" for these events
                'eventdate': row['cancellationrejectiondateutc'],
                'leadid': row['id'],
                'updateddateutc': row['updateddateutc']
            })

    return pd.DataFrame(events)

In [78]:
print(df_leads.columns)

df_events = transform_data(df_leads)

Index(['id', 'state', 'createddateutc', 'cancellationrequestdateutc',
       'cancellationdateutc', 'cancellationrejectiondateutc', 'soldemployee',
       'cancelledemployee', 'updateddateutc'],
      dtype='object')


In [80]:

# Insert transformed data back to Snowflake
df_events.to_sql('leadevents', con=engine, index=False, if_exists='append')
print("Data loaded successfully.")

Data loaded successfully.


In [9]:
# Close connection
connection.close()